In [1]:
print("Hello World")

Hello World


In [2]:
from sklearn.datasets import load_files       
from keras.utils import np_utils
import numpy as np
from glob import glob

# define function to load train, test, and validation datasets
def load_dataset(path):
    data = load_files(path)
    skin_files = np.array(data['filenames'])
    skin_targets = np_utils.to_categorical(np.array(data['target']), 133)
    return skin_files, skin_targets

# load train, test, and validation datasets
train_files, train_targets = load_dataset('data/train')
valid_files, valid_targets = load_dataset('data/valid')
test_files, test_targets = load_dataset('data/test')

# load list of skin names
skin_names = [item[20:-1] for item in sorted(glob("data/train/*/"))]

# print statistics about the dataset
print('There are %d total skin categories.' % len(skin_names))
print('There are %s total skin images.\n' % len(np.hstack([train_files, valid_files, test_files])))
print('There are %d training skin images.' % len(train_files))
print('There are %d validation skin images.' % len(valid_files))
print('There are %d test skin images.'% len(test_files))

Using TensorFlow backend.


There are 3 total skin categories.
There are 2750 total skin images.

There are 2000 training skin images.
There are 150 validation skin images.
There are 600 test skin images.



## Step 1: Detect Human Eyes

We use OpenCV's implementation of Haar feature-based cascade classifiers to detect human faces in images. OpenCV provides many pre-trained face detectors, stored as XML files on github. We have downloaded one of these detectors and stored it in the haarcascades directory.
In the next code cell, we demonstrate how to use this detector to find human faces in a sample image.

In [15]:
import cv2                
import matplotlib.pyplot as plt                        
%matplotlib inline                               

# load filenames in shuffled human dataset
human_files = np.array(glob("data/eyes/*"))

# extract pre-trained eye detector
eye_cascade = cv2.CascadeClassifier('haarcascades/haarcascade_lefteye_2splits.xml')

# load color (BGR) image
img = cv2.imread(human_files[:100])

# it is standard procedure to convert the images to grayscale
# convert BGR image to grayscale
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

# find eyes in image
eyes = eye_cascade.detectMultiScale(gray)

# print number of eyes detected in the image
print('Number of eyes detected:', len(eyes))

# get bounding box for each detected eye
for (x,y,w,h) in eyes:
    # add bounding box to color image
    cv2.rectangle(img,(x,y),(x+w,y+h),(255,0,0),2)
    
# convert BGR image to RGB for plotting
cv_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

# display the image, along with bounding box
plt.imshow(cv_rgb)
plt.show()

TypeError: bad argument type for built-in operation

## Crop Eyes

In [10]:
import os
import sys

num = 0

for (x,y,w,h) in eyes:
    crop_img = img[y:y+h, x:x+w]
    path = 'data/crop_eyes'
    cv2.imwrite(os.path.join(path , 'image{}.jpg'.format(num)), crop_img)
    num += 1


We can use this procedure to write a function that returns True if a human face is detected in an image and False otherwise. This function, aptly named face_detector, takes a string-valued file path to an image as input and appears in the code block below.

In [ ]:
# returns "True" if face is detected in image stored at img_path
def face_detector(img_path):
    img = cv2.imread(img_path)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    eyes = eye_cascade.detectMultiScale(gray)
    return len(eyes) > 0